# Creating a Funding Transaction

In this section we'll create a Lightning Channel funding transaction from scratch in python. We'll go through each part of the transaction, how it's constructed, signed, and the message the peers exchange to send and get the needed information from each other to make it happen. We'll test it using bitcoin core in regtest mode.

## Prerequisite knowledge
- For all notebooks:
    - A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook) by Andreas Antonopoulos UTXO model, in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
    - A high level understanding of the Lightning Network. e.g. [Mastering Lightning](https://github.com/bitcoinbook/bitcoinbook) TODO Get the chapter to point it here
    - A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
    - [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).

- Specific to this notebook:
    - SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
    - Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
    - Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
    - Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'

## Channel Establishment v1

These Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/00-introduction.md)) defines two pathways to create a channel, lets focus on the legacy one because most of the channels are created using it, and lets the 'Channel Establishment v2' for the following notebook.

    +-----------+                              +---------+
    |           |--(1)---  open_channel  ----->|         |
    |           |<-(2)--  accept_channel  -----|         |
    |           |                              |         |
    |   Alice   |--(3)--  funding_created  --->|   Bob   |
    |           |<-(4)--  funding_signed  -----|         |
    |           |                              |         |
    |           |--(5)---  channel_ready  ---->|         |
    |           |<-(6)---  channel_ready  -----|         |
    +-----------+                              +---------+

    - where node Alice is 'funder' and node Bob is 'fundee'



Alice send the [`open_channel`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#the-open_channel-message) message to Bob. This message define many channel operational criterias, here we are going to focus on the information that are used in the funding transaction:
* chain_hash - denotes the exact blockchain that the opened channel will reside within.
* funding_satoshis - is the amount the sender is putting into the channel
* push_msat - is an amount of initial funds that the sender is unconditionally giving to the receiver.
* funding_pubkey - is the public key in the 2-of-2 multisig script of the funding transaction output.


If Bob does not agree with the criterias sent into the `open_channel` message he send a [`error_message`](https://github.com/lightning/bolts/blob/master/01-messaging.md#the-error-and-warning-messages) back to Alice, otherwise he sends the [`accept_channel`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#the-accept_channel-message) message, containing some operational criterias from your side, here we are going to focus on the information that are used in the funding transaction:
* funding_pubkey - is the public key in the 2-of-2 multisig script of the funding transaction output.

## The Funding Transaction

The funding transaction input could be one or more UTXOs from Alice (the funder) and the output arethea change to Alice and a P2WSH output (pay-to-witness-script-hash)<sup>[BIP141](https://github.com/bitcoin/bips/blob/master/bip-0141.mediawiki#witness-program)</sup> funding the channel, the order of the pubkeys on the output script is defined lexicographically, where pubkey1 is the lesser of the two funding_pubkey in compressed format and pubkey2 is the greater.

`2 <pubkey1> <pubkey2> 2 OP_CHECKMULTISIG`

## Setup 

### Requirements
For this exercise we'll need Bitcoin Core. This notebook has been tested with [v24.0.1](https://github.com/bitcoin/bitcoin/releases/tag/v24.0.1).

Below, set the paths for:
1. The bitcoin core functional test framework directory.
2. The directory containing lightning-tx-tutorial.

**You'll need to edit these next two lines for your local setup.**

In [2]:
path_to_bitcoin_functional_test = "/home/pins-dev/Projects/bitcoin/test/functional"
path_to_bitcoin_tx_tutorial = "/home/pins-dev/Projects/lightning-tx-tutorial"

import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the lightning-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_bitcoin_tx_tutorial)
from functions import *

import json